IMPORT THU VIEN

In [1]:
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import numpy as np
import time
from selenium import webdriver
from bs4 import BeautifulSoup
import csv

KHOI TAO DRIVER CHO TRINH DUYET


In [2]:
driver = webdriver.Chrome('D:\chromedriver.exe')
url = 'https://www.chotot.com/mua-ban-dien-thoai-apple-cu-sdmd1ec3'
driver.get(url)
time.sleep(3)
crawl_number=10

C:\Users\ACER\AppData\Local\Temp\ipykernel_44576\2825582994.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('D:\chromedriver.exe')


HÀM CHÍNH

cuộn trang

In [3]:
def scroll_page():
    SCROLL_PAUSE_TIME = 0.5  # Thời gian dừng giữa các lần cuộn
    SCROLL_STEP = 250  # Bước cuộn

    # Lấy chiều cao của cửa sổ trình duyệt
    window_height = driver.execute_script("return window.innerHeight")

    # Lấy chiều cao của toàn bộ trang web
    document_height = driver.execute_script("return document.documentElement.scrollHeight")

    # Thực hiện cuộn dần dần cho đến khi cuộn hết nội dung
    while True:
        # Cuộn trang
        driver.execute_script(f"window.scrollBy(0, {SCROLL_STEP})")
        
        # Dừng một lúc để cho trang tải nội dung mới
        time.sleep(SCROLL_PAUSE_TIME)
        
        # Lấy chiều cao đã cuộn đến
        scrolled_height = driver.execute_script("return window.pageYOffset")
        
        # Kiểm tra nếu đã cuộn hết nội dung
        if (scrolled_height + window_height) >= document_height:
            break


CRAWL THEO LINK SAN PHAM

In [4]:
def get_product_one_page(data_number):
    current_page_url = driver.current_url
    print(current_page_url)
    page_source = BeautifulSoup(driver.page_source)
    
    name_result = page_source.find_all('h3', class_ = 'commonStyle_adTitle__g520j')
    price_result = page_source.find_all('p', class_ = 'AdBody_adPriceNormal___OYFU')
    list_names = []
    list_price = []
    if(data_number>20):
        data_number = 20
    else:
        data_number = data_number
    
    for item in range(len(name_result)):
            name_item = name_result[item].text
            # location_item = locate_result[item].text
            price_item = price_result[item].text
            
            list_names.append(name_item)
            list_price.append(price_item)
            # list_location.append(price_item)
            item = item+1
            if(item == data_number):
                break
    scroll_page()
    #shop/nguoi
    elements = driver.find_elements(By.CSS_SELECTOR, '.AdItemFooter_wrapperFooter__myuTA')
    list_results = []
    count1=1
    for element in elements:
        img_element = element.find_element(By.TAG_NAME, 'img')
        src = img_element.get_attribute('src')
        if src == "https://static.chotot.com/storage/icons/owner/shop.svg":
            list_results.append(1)
        else:
            list_results.append(0)
        if(count1==data_number):
            break
        count1=count1+1    
    #lay thong tin tung link
    
    link_list = []
    elements = driver.find_elements(By.CSS_SELECTOR,".AdItem_adItem__gDDQT")
    # Lặp qua từng phần tử để lấy giá trị href
    count=1
    for element in elements:
        href = element.get_attribute("href")
        link_list.append(href)
        if(count==data_number):
            break
        count=count+1
    
    #tu link lay thong tin 
    brand_list=[]
    condition_list=[]
    type_list=[]
    preserve_list=[]
    storage_list=[]
    for link in link_list:
    # Truy cập vào liên kết
    #brand
        driver.get(link)
        page_source = BeautifulSoup(driver.page_source,"html.parser")
        brand_result = page_source.find('span', attrs={'itemprop': 'mobile_brand'})
        if not brand_result:
            brand_list.append(None)
        else:
            brand_list.append(brand_result.text)
    #condition
        page_source = BeautifulSoup(driver.page_source,"html.parser")
        condition_result = page_source.find('span', attrs={'itemprop': 'elt_condition'})
        if not condition_result:
            condition_list.append(None)
        else:
            condition_list.append(condition_result.text)
    #type   
        page_source = BeautifulSoup(driver.page_source,"html.parser")
        type_result = page_source.find('span', attrs={'itemprop': 'mobile_model'})
        if not type_result:
            type_list.append(None)
        else:
            type_list.append(type_result.text)
    #preserve
        page_source = BeautifulSoup(driver.page_source,"html.parser")
        preserve_result = page_source.find('span', attrs={'itemprop': 'elt_warranty'})
        if not preserve_result:
            preserve_list.append(None)
        else:
            preserve_list.append(preserve_result.text)
    #storage
        page_source = BeautifulSoup(driver.page_source,"html.parser")
        storage_result = page_source.find('span', attrs={'itemprop': 'mobile_capacity'})
        if not storage_result:
            storage_list.append(None)
        else:
            storage_list.append(storage_result.text)
    #tra ve du lieu
    df = pd.DataFrame()
    df['names'] = list_names
    df['price'] = list_price
    df['link'] = link_list
    df['brand'] = brand_list
    df['type'] = type_list
    df['condition'] = condition_list
    df['preserve'] = preserve_list
    df['storage'] = storage_list
    df['result'] = list_results
    driver.get(current_page_url)  
    print(df)      
    return df

CRAWL NHIEU TRANG

In [5]:
def get_product_all_pages(number_of_page,data_number):
    all_page = pd.DataFrame()
    
    number_of_pages = number_of_page
    i = 1
    for page in range(number_of_pages):
        print('số trang: '+str(i))
        i=i+1
        one_page = get_product_one_page(data_number)
        all_page = all_page.append(one_page)
        time.sleep(1)
        
#         scroll the page down to the "next" button
        scroll_page()
        scroll_page()
        next_button = driver.find_element(By.CSS_SELECTOR, '.Paging_rightIcon__3p8MS')
        if(page!=number_of_pages-1):
            driver.execute_script("arguments[0].click();", next_button)    
        data_number = data_number - 20    
        time.sleep(1)
    return all_page

CRAWL THEO SO MAU

In [6]:
def crawl_data_number(data_number):
    df=pd.DataFrame()
    page = int(data_number / 20)+1
    df=get_product_all_pages(page,data_number)
    return(df)

Crawl 5 mau iphone va luu vao df

In [7]:
df=pd.DataFrame()
df=crawl_data_number(crawl_number/2)


số trang: 1
https://www.chotot.com/mua-ban-dien-thoai-apple-cu-sdmd1ec3
                                               names         price  \
0                Iphone 12 quốc tế 128gb có giao lưu   7.300.000 đ   
1  Cần tiền đóng trọ bay 11 Pro max lock vàng hoặ...   7.000.000 đ   
2  iPhone 12 Pro || 12Promax quốc tế zin ốc - GÓP 0đ  11.700.000 đ   
3     iphone Xr Nguyên Zin Áp Chưa Bung 100%(Góp 0%)   4.990.000 đ   
4                     IPhone 7 32GB pin 90% đẹp keng   1.200.000 đ   

                                                link  brand  \
0  https://www.chotot.com/mua-ban-dien-thoai-than...  Apple   
1  https://www.chotot.com/mua-ban-dien-thoai-quan...  Apple   
2  https://www.chotot.com/mua-ban-dien-thoai-quan...  Apple   
3  https://www.chotot.com/mua-ban-dien-thoai-quan...  Apple   
4  https://www.chotot.com/mua-ban-dien-thoai-thi-...  Apple   

                type                   condition       preserve storage  \
0          iPhone 12   Đã sử dụng (qua sửa chữa)     

C:\Users\ACER\AppData\Local\Temp\ipykernel_44576\4133581705.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_page = all_page.append(one_page)


crawl 5 mau Samsung

In [8]:

url = 'https://www.chotot.com/mua-ban-dien-thoai-samsung-cu-sdmd2ec3'
driver.get(url)
time.sleep(3)

In [9]:
df1=pd.DataFrame()
df1=crawl_data_number(crawl_number/2)

số trang: 1
https://www.chotot.com/mua-ban-dien-thoai-samsung-cu-sdmd2ec3
                                               names        price  \
0                              Note 10 Plus Mỹ 512GB  6.500.000 đ   
1         Samsung Galaxy S21 FE 5G USA 128G snap 888  5.990.000 đ   
2  Điện Thoại Samsung Galaxy Note9 2 Sim Ram8Gb/5...  3.750.000 đ   
3                     S20Ultra 5G Snap 2sim - Góp 0Đ  7.790.000 đ   
4                           màn zin đẹp keng ko ám ố  1.600.000 đ   

                                                link    brand  \
0  https://www.chotot.com/mua-ban-dien-thoai-quan...  Samsung   
1  https://www.chotot.com/mua-ban-dien-thoai-quan...  Samsung   
2  https://www.chotot.com/mua-ban-dien-thoai-quan...  Samsung   
3  https://www.chotot.com/mua-ban-dien-thoai-quan...  Samsung   
4  https://www.chotot.com/mua-ban-dien-thoai-quan...  Samsung   

                  type                   condition       preserve   storage  \
0  Galaxy Note 10 Plus   Đã sử dụng (qua 

C:\Users\ACER\AppData\Local\Temp\ipykernel_44576\4133581705.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_page = all_page.append(one_page)


Tong hop 2 df ta duoc 10 mau du lieu

In [10]:
df=pd.concat([df, df1])

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 4
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   names      10 non-null     object
 1   price      10 non-null     object
 2   link       10 non-null     object
 3   brand      10 non-null     object
 4   type       10 non-null     object
 5   condition  10 non-null     object
 6   preserve   10 non-null     object
 7   storage    7 non-null      object
 8   result     10 non-null     int64 
dtypes: int64(1), object(8)
memory usage: 800.0+ bytes


names: ten 

price: gia 

link: link san pham

brand: samsung/iphone

type: loai san pham

condition:  tinh trang san pham

preserve: tinh trang bao hanh

storage: dung luong

result: duoc ban boi ca nhan hay cua hang chuyen nghiep


Chuyen chu hoa ve chu thuong de dong bo du lieu

In [12]:
df['brand'] = df['brand'].str.lower()
df['type'] = df['type'].str.lower()
df['condition'] = df['condition'].str.lower()
df['preserve'] = df['preserve'].str.lower()
df['storage'] = df['storage'].str.lower()

In [13]:
df.head(10)

,names,price,link,brand,type,condition,preserve,storage,result
0,Iphone 12 quốc tế 128gb có giao lưu,7.300.000 đ,https://www.chotot.com/mua-ban-dien-thoai-than...,apple,iphone 12,đã sử dụng (qua sửa chữa),1 tháng,128 gb,1
1,Cần tiền đóng trọ bay 11 Pro max lock vàng hoặ...,7.000.000 đ,https://www.chotot.com/mua-ban-dien-thoai-quan...,apple,iphone 11 pro max,đã sử dụng (chưa sửa chữa),đang cập nhật,64 gb,0
2,iPhone 12 Pro || 12Promax quốc tế zin ốc - GÓP 0đ,11.700.000 đ,https://www.chotot.com/mua-ban-dien-thoai-quan...,apple,iphone 12 pro,đã sử dụng (chưa sửa chữa),đang cập nhật,None,1
3,iphone Xr Nguyên Zin Áp Chưa Bung 100%(Góp 0%),4.990.000 đ,https://www.chotot.com/mua-ban-dien-thoai-quan...,apple,iphone xr,đã sử dụng (chưa sửa chữa),còn bảo hành,64 gb,1
4,IPhone 7 32GB pin 90% đẹp keng,1.200.000 đ,https://www.chotot.com/mua-ban-dien-thoai-thi-...,apple,iphone 7,đã sử dụng (chưa sửa chữa),hết bảo hành,32 gb,0
0,Note 10 Plus Mỹ 512GB,6.500.000 đ,https://www.chotot.com/mua-ban-dien-thoai-quan...,samsung,galaxy note 10 plus,đã sử dụng (qua sửa chữa),1 tháng,> 256 gb,0
1,Samsung Galaxy S21 FE 5G USA 128G snap 888,5.990.000 đ,https://www.chotot.com/mua-ban-dien-thoai-quan...,samsung,galaxy s21 fe,đã sử dụng (chưa sửa chữa),còn bảo hành,128 gb,1
2,Điện Thoại Samsung Galaxy Note9 2 Sim Ram8Gb/5...,3.750.000 đ,https://www.chotot.com/mua-ban-dien-thoai-quan...,samsung,galaxy note 9,đã sử dụng (chưa sửa chữa),3 tháng,> 256 gb,0
3,S20Ultra 5G Snap 2sim - Góp 0Đ,7.790.000 đ,https://www.chotot.com/mua-ban-dien-thoai-quan...,samsung,galaxy s20 ultra,đã sử dụng (chưa sửa chữa),đang cập nhật,None,1
4,màn zin đẹp keng ko ám ố,1.600.000 đ,https://www.chotot.com/mua-ban-dien-thoai-quan...,samsung,galaxy a51,đã sử dụng (chưa sửa chữa),đang cập nhật,None,0


dem so loai du lieu va so luong cua type

In [14]:
df['type'].value_counts()


iphone 12              1
iphone 11 pro max      1
iphone 12 pro          1
iphone xr              1
iphone 7               1
galaxy note 10 plus    1
galaxy s21 fe          1
galaxy note 9          1
galaxy s20 ultra       1
galaxy a51             1
Name: type, dtype: int64

luu type vao dfas sau khi loai bo trung

In [15]:
dfas=df['type'].drop_duplicates()


In [16]:
list_type=[]
list_type=dfas.tolist()
list_type=list(set(list_type))
print(len(list_type))

10


Thuc hien crawl web gsmarena

In [17]:
driver = webdriver.Chrome()

# Mở trang web
driver.get("https://www.gsmarena.com/")

In [18]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import urllib.parse


Ham crawl ( nhap du lieu , tim kiem du lieu)

In [19]:
def get_info_type_phone(k,l):
    camera_list=[]
    display_list=[]
    # ram_list=[]
    pin_list=[]
    type_list=[]
    tag_list=[]
    date_list=[]
    ram_storage_list=[]
    count=0
    page_source = BeautifulSoup(driver.page_source,"html.parser")
    for i in range(k,l):
        print(i)
        type_list.append(list_type[i])
        print(list_type[i])
        search_input =driver.find_element(By.CSS_SELECTOR,"input#topsearch-text")
        search_input.clear()  # Xóa nội dung hiện có trong ô tìm kiếm (nếu có)
        search_input.send_keys(list_type[i])  # Nhập tên hãng vào ô tìm kiếm
        search_input.send_keys(Keys.RETURN)
        elements = driver.find_elements(By.CSS_SELECTOR, 'div.makers a')
        # print(len(elements))
        elementnew = driver.find_elements(By.CSS_SELECTOR, '.makers')

        for element in elementnew:
            no_results = element.find_elements(By.CSS_SELECTOR, 'div.no-results')
            
        no_results2 = driver.find_elements(By.CSS_SELECTOR, 'h3.section')
        if len(no_results2) > 0:
            no_results_text = no_results2[0].text
        else:
            no_results_text = "hi"
        print(len(no_results))
        if(len(no_results)==0) and ("We're sorry, we found no results" not in no_results_text):
            for element in elements:
                link = element.get_attribute('href')  # Trích xuất giá trị thuộc tính href của phần tử <a>
                print(link)
                time.sleep(5)
                driver.get(link)
                display=driver.find_element(By.CSS_SELECTOR,"li.help.accented.help-display strong.accent")
                if display: 
                    display_list.append(display.text)
                    print(display_list)
                else: 
                    display_list.append(None)
                td_element = driver.find_element(By.CSS_SELECTOR, "td[data-spec=internalmemory]")
                if td_element:
                    text = td_element.text
                    ram_storage_list.append(text)
                    print(ram_storage_list)
                else:
                    ram_storage_list.append(None)
                spanElement = driver.find_element(By.CSS_SELECTOR,"span[data-spec=released-hl]")
                if spanElement:  
                    text = spanElement.text
                    date_list.append(text)
                    print(date_list)
                else:
                    date_list.append(None)
        
                camera=driver.find_element(By.CSS_SELECTOR,"strong.accent.accent-camera")
                if camera: 
                    camera_list.append(camera.text)
                    print(camera_list)
                else: 
                    camera_list.append(None)
                # ram=driver.find_element(By.CSS_SELECTOR,"strong.accent.accent-expansion")
                # if ram: 
                #     ram_list.append(ram.text)
                #     print(ram_list)
                # else: 
                #     ram_list.append(None)
                pin=driver.find_element(By.CSS_SELECTOR,"strong.accent.accent-battery")
                if pin: 
                    pin_list.append(pin.text)
                    print(pin_list)
                else: 
                    pin_list.append(None)
                tag=driver.find_element(By.CSS_SELECTOR,"h1.specs-phone-name-title")
                if tag: 
                    tag_list.append(tag.text)
                    print(tag_list)
                else: 
                    tag_list.append(None)
                break
        else:
            display_list.append(None)
            camera_list.append(None)
            # ram_list.append(None)
            pin_list.append(None)
            tag_list.append(None)
            date_list.append(None)
            ram_storage_list.append(None)
            print("hi")
    df = pd.DataFrame()
    df['type'] = type_list
    df['display'] = display_list
    df['camera'] = camera_list
    # df['ram'] = ram_list
    df['pin'] = pin_list
    df['tag'] = tag_list
    df['date'] = date_list
    df['ram_storage'] = ram_storage_list
    return df

In [20]:
df1= pd.DataFrame() 
df1=get_info_type_phone(0,len(list_type))  
df1.head()

0
galaxy note 10 plus
0
https://www.gsmarena.com/samsung_galaxy_note10+-9732.php
['6.8"']
['256GB 12GB RAM, 512GB 12GB RAM']
['Released 2019, August 23']
['12MP']
['4300mAh']
['Samsung Galaxy Note10+']
1
iphone 12
0
https://www.gsmarena.com/apple_iphone_12-10509.php
['6.8"', '6.1"']
['256GB 12GB RAM, 512GB 12GB RAM', '64GB 4GB RAM, 128GB 4GB RAM, 256GB 4GB RAM']
['Released 2019, August 23', 'Released 2020, October 23']
['12MP', '12MP']
['4300mAh', '2815mAh']
['Samsung Galaxy Note10+', 'Apple iPhone 12']
2
iphone 11 pro max
0
https://www.gsmarena.com/apple_iphone_11_pro_max-9846.php
['6.8"', '6.1"', '6.5"']
['256GB 12GB RAM, 512GB 12GB RAM', '64GB 4GB RAM, 128GB 4GB RAM, 256GB 4GB RAM', '64GB 4GB RAM, 256GB 4GB RAM, 512GB 4GB RAM']
['Released 2019, August 23', 'Released 2020, October 23', 'Released 2019, September 20']
['12MP', '12MP', '12MP']
['4300mAh', '2815mAh', '3969mAh']
['Samsung Galaxy Note10+', 'Apple iPhone 12', 'Apple iPhone 11 Pro Max']
3
galaxy s20 ultra
0
https://www.gsmar

,type,display,camera,pin,tag,date,ram_storage
0,galaxy note 10 plus,"6.8""",12MP,4300mAh,Samsung Galaxy Note10+,"Released 2019, August 23","256GB 12GB RAM, 512GB 12GB RAM"
1,iphone 12,"6.1""",12MP,2815mAh,Apple iPhone 12,"Released 2020, October 23","64GB 4GB RAM, 128GB 4GB RAM, 256GB 4GB RAM"
2,iphone 11 pro max,"6.5""",12MP,3969mAh,Apple iPhone 11 Pro Max,"Released 2019, September 20","64GB 4GB RAM, 256GB 4GB RAM, 512GB 4GB RAM"
3,galaxy s20 ultra,"6.9""",108MP,5000mAh,Samsung Galaxy S20 Ultra 5G,"Released 2020, March 06","128GB 12GB RAM, 256GB 12GB RAM, 512GB 16GB RAM"
4,galaxy note 9,"6.4""",12MP,4000mAh,Samsung Galaxy Note9,"Released 2018, August 24","128GB 6GB RAM, 128GB 8GB RAM, 512GB 8GB RAM"


Merge data cu va data moi theo type

In [21]:
merged_df = pd.merge(df, df1, on='type')

In [22]:
merged_df.head(10)

,names,price,link,brand,type,condition,preserve,storage,result,display,camera,pin,tag,date,ram_storage
0,Iphone 12 quốc tế 128gb có giao lưu,7.300.000 đ,https://www.chotot.com/mua-ban-dien-thoai-than...,apple,iphone 12,đã sử dụng (qua sửa chữa),1 tháng,128 gb,1,"6.1""",12MP,2815mAh,Apple iPhone 12,"Released 2020, October 23","64GB 4GB RAM, 128GB 4GB RAM, 256GB 4GB RAM"
1,Cần tiền đóng trọ bay 11 Pro max lock vàng hoặ...,7.000.000 đ,https://www.chotot.com/mua-ban-dien-thoai-quan...,apple,iphone 11 pro max,đã sử dụng (chưa sửa chữa),đang cập nhật,64 gb,0,"6.5""",12MP,3969mAh,Apple iPhone 11 Pro Max,"Released 2019, September 20","64GB 4GB RAM, 256GB 4GB RAM, 512GB 4GB RAM"
2,iPhone 12 Pro || 12Promax quốc tế zin ốc - GÓP 0đ,11.700.000 đ,https://www.chotot.com/mua-ban-dien-thoai-quan...,apple,iphone 12 pro,đã sử dụng (chưa sửa chữa),đang cập nhật,None,1,"6.7""",12MP,3687mAh,Apple iPhone 12 Pro Max,"Released 2020, November 13","128GB 6GB RAM, 256GB 6GB RAM, 512GB 6GB RAM"
3,iphone Xr Nguyên Zin Áp Chưa Bung 100%(Góp 0%),4.990.000 đ,https://www.chotot.com/mua-ban-dien-thoai-quan...,apple,iphone xr,đã sử dụng (chưa sửa chữa),còn bảo hành,64 gb,1,"6.1""",12MP,2942mAh,Apple iPhone XR,"Released 2018, October 26","64GB 3GB RAM, 128GB 3GB RAM, 256GB 3GB RAM"
4,IPhone 7 32GB pin 90% đẹp keng,1.200.000 đ,https://www.chotot.com/mua-ban-dien-thoai-thi-...,apple,iphone 7,đã sử dụng (chưa sửa chữa),hết bảo hành,32 gb,0,"4.7""",12MP,1960mAh,Apple iPhone 7,"Released 2016, September 16","32GB 2GB RAM, 128GB 2GB RAM, 256GB 2GB RAM"
5,Note 10 Plus Mỹ 512GB,6.500.000 đ,https://www.chotot.com/mua-ban-dien-thoai-quan...,samsung,galaxy note 10 plus,đã sử dụng (qua sửa chữa),1 tháng,> 256 gb,0,"6.8""",12MP,4300mAh,Samsung Galaxy Note10+,"Released 2019, August 23","256GB 12GB RAM, 512GB 12GB RAM"
6,Samsung Galaxy S21 FE 5G USA 128G snap 888,5.990.000 đ,https://www.chotot.com/mua-ban-dien-thoai-quan...,samsung,galaxy s21 fe,đã sử dụng (chưa sửa chữa),còn bảo hành,128 gb,1,"6.4""",12MP,4500mAh,Samsung Galaxy S21 FE 5G,"Released 2022, January 07","128GB 6GB RAM, 128GB 8GB RAM, 256GB 6GB RAM, 2..."
7,Điện Thoại Samsung Galaxy Note9 2 Sim Ram8Gb/5...,3.750.000 đ,https://www.chotot.com/mua-ban-dien-thoai-quan...,samsung,galaxy note 9,đã sử dụng (chưa sửa chữa),3 tháng,> 256 gb,0,"6.4""",12MP,4000mAh,Samsung Galaxy Note9,"Released 2018, August 24","128GB 6GB RAM, 128GB 8GB RAM, 512GB 8GB RAM"
8,S20Ultra 5G Snap 2sim - Góp 0Đ,7.790.000 đ,https://www.chotot.com/mua-ban-dien-thoai-quan...,samsung,galaxy s20 ultra,đã sử dụng (chưa sửa chữa),đang cập nhật,None,1,"6.9""",108MP,5000mAh,Samsung Galaxy S20 Ultra 5G,"Released 2020, March 06","128GB 12GB RAM, 256GB 12GB RAM, 512GB 16GB RAM"
9,màn zin đẹp keng ko ám ố,1.600.000 đ,https://www.chotot.com/mua-ban-dien-thoai-quan...,samsung,galaxy a51,đã sử dụng (chưa sửa chữa),đang cập nhật,None,0,"6.5""",48MP,4000mAh,Samsung Galaxy A51,"Released 2019, December 16","64GB 4GB RAM, 128GB 4GB RAM, 128GB 6GB RAM, 12..."


In [23]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 9
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   names        10 non-null     object
 1   price        10 non-null     object
 2   link         10 non-null     object
 3   brand        10 non-null     object
 4   type         10 non-null     object
 5   condition    10 non-null     object
 6   preserve     10 non-null     object
 7   storage      7 non-null      object
 8   result       10 non-null     int64 
 9   display      10 non-null     object
 10  camera       10 non-null     object
 11  pin          10 non-null     object
 12  tag          10 non-null     object
 13  date         10 non-null     object
 14  ram_storage  10 non-null     object
dtypes: int64(1), object(14)
memory usage: 1.2+ KB


ta duoc du lieu cuoi cung gom 15 truong

them 1 so truong moi:

display: thong so man hinh

camera: thong so cam

pin: dung luong pin

tag: ten san pham duoc search 

date: ngay xuat hien

ram_storage: ram va dung luong tuong ung